In [4]:
import numpy as np
import numpy.linalg as la
from scipy import linalg as scila
from func import *

np.set_printoptions(linewidth=200)

### Toom-Cook Generate

In [5]:
def toomCookMats(b, n, cheby=False):
    pts = rs(n+b-1)
    V = np.vander(pts,increasing=True)
    V[-1,:] = 0; V[-1,-1] = 1
    C = la.inv(V)
    
    A = V[:,:b].copy(); A[-1,-1] = 1
    B = V[:,:n].copy(); B[-1,-1] = 1
    
    return [C,A,B]

In [6]:
A,B,C = toomCookMats(3,4)

print(A,"\n\n",B,"\n\n",C)

[[ 1.          0.          0.          0.          0.          0.        ]
 [-0.         -0.66666667  0.66666667  0.08333333 -0.08333333  4.        ]
 [-1.25        0.66666667  0.66666667 -0.04166667 -0.04166667  0.        ]
 [ 0.          0.16666667 -0.16666667 -0.08333333  0.08333333 -5.        ]
 [ 0.25       -0.16666667 -0.16666667  0.04166667  0.04166667 -0.        ]
 [ 0.          0.          0.          0.          0.          1.        ]] 

 [[ 1  0  0]
 [ 1 -1  1]
 [ 1  1  1]
 [ 1 -2  4]
 [ 1  2  4]
 [ 0  0  1]] 

 [[ 1  0  0  0]
 [ 1 -1  1 -1]
 [ 1  1  1  1]
 [ 1 -2  4 -8]
 [ 1  2  4  8]
 [ 0  0  0  1]]


### FFT Generate

In [7]:
def fftMats(b, n, cyclic=False):
    assert(b <= n)
    z = (n if cyclic else n+b-1)
    F = Fmat(z)
    Finv = Fmatinv(z)
    
    return [F[:,:n].copy(),F[:,:b].copy(),Finv]

In [8]:
A,B,C = fftMats(3,4)

print(A,"\n\n",B,"\n\n",C)

[[ 1. +0.00000000e+00j  1. +0.00000000e+00j  1. +0.00000000e+00j  1. +0.00000000e+00j]
 [ 1. +0.00000000e+00j  0.5-8.66025404e-01j -0.5-8.66025404e-01j -1. -3.88578059e-16j]
 [ 1. +0.00000000e+00j -0.5-8.66025404e-01j -0.5+8.66025404e-01j  1. +8.32667268e-16j]
 [ 1. +0.00000000e+00j -1. -3.88578059e-16j  1. +8.32667268e-16j -1. -1.33226763e-15j]
 [ 1. +0.00000000e+00j -0.5+8.66025404e-01j -0.5-8.66025404e-01j  1. +1.77635684e-15j]
 [ 1. +0.00000000e+00j  0.5+8.66025404e-01j -0.5+8.66025404e-01j -1. -2.16493490e-15j]] 

 [[ 1. +0.00000000e+00j  1. +0.00000000e+00j  1. +0.00000000e+00j]
 [ 1. +0.00000000e+00j  0.5-8.66025404e-01j -0.5-8.66025404e-01j]
 [ 1. +0.00000000e+00j -0.5-8.66025404e-01j -0.5+8.66025404e-01j]
 [ 1. +0.00000000e+00j -1. -3.88578059e-16j  1. +8.32667268e-16j]
 [ 1. +0.00000000e+00j -0.5+8.66025404e-01j -0.5-8.66025404e-01j]
 [ 1. +0.00000000e+00j  0.5+8.66025404e-01j -0.5+8.66025404e-01j]] 

 [[ 0.16666667+0.00000000e+00j  0.16666667+0.00000000e+00j  0.16666667+0.00

In [9]:
from sympy import Poly
from sympy.polys import ring, QQ
RR, x = ring("x", QQ)

### Extended Euclian Algorithm

Code is very ad-hoc right now. Need extended Euclidian algorithm from sympy.

In [10]:
def sym_deg(p):
    return p.terms()[0][0][0]

def sym_allcoeffs(p):
    # extracts all coefficients form 1 -> x**p
    d = sym_deg(p)
    coeffs = np.zeros(d+1)
    terms = p.to_dict()
    for i in range(d+1):
        v = tuple([i])
        if v in terms:
            coeffs[i] = terms[v]
    return coeffs

def extEucAlg(M,m,RR,deg):
    # returns coefficients of N(x) and n(x) 
    [N,n,gcd] = RR.dup_gcdex(M, m)
    assert(len(gcd) == 1 and gcd.coeffs()[0] == 1)
    # !!! <!-- TODO --> sum of degrees not right, this is an ad-hoc fix
    if(sym_deg(M) + sym_deg(N) != deg):
        Ntemp = N
        N = n*m
        n = Ntemp*M
    return [sym_allcoeffs(N),sym_allcoeffs(n)]

def berzotMats(pTerms,numCoeffs):
    deg = len(pTerms)-1
    E = np.zeros((numCoeffs + deg, numCoeffs))
    for i in range(deg):
        for j in range(deg+1):
            E[j+i,i] = pTerms[j]
    return E

In [11]:
M = x**2+1
m = x**2-1
N,n = extEucAlg(M,m,RR,4)

print(N)
print(n)

Nmat = berzotMats(N,2)
nmat = berzotMats(n,2)
print(Nmat)
print(nmat)

[ 0.5  0.  -0.5]
[0.5 0.  0.5]
[[ 0.5  0. ]
 [ 0.   0.5]
 [-0.5  0. ]
 [ 0.  -0.5]]
[[0.5 0. ]
 [0.  0.5]
 [0.5 0. ]
 [0.  0.5]]


Composition of Winograd's algorithm

$$
    \begin{bmatrix}E_1 & E_2 \end{bmatrix}
    \begin{bmatrix}M_3C_1 & 0 \\ 0 & M_4C_2 \end{bmatrix}
    \Bigg(
        \begin{bmatrix} A_1M_1\\A_2M_2 \end{bmatrix}f
        \odot
        \begin{bmatrix} B_1M_1\\B_2M_2 \end{bmatrix}g
    \Bigg)
$$

Matrices $M_3$ and $M_4$ are defined by the companion matrix, $M_1$ and $M_2$ are companion like matrices, staggered values.

### Generating Cyclic Convolutions

We generate a 4 point, modulo $M(x) = x^4-1 = (x^2+1)(x^2-1)$.

#### Building Modulo Matrices

We can represent our original polynomial with a $(d+1) \times (d+1)$, where the $i^{th}$ row corresponds to the coefficient $x^{i-1}$. Within each row, the corresponding columns represent the coefficient of the original variable. For example, given a polynomial with degree $d_1$, $c_3x^3 + c_2x^2 + c_1x + c_0$, we build the identity matrix since the $i^{th}$ coefficients start off only at the $(i-1)^{th}$ degree.

Let the divisor be of degree $d_2$ be $x^2 + 2$. We build the column vector

$$\begin{bmatrix}2 \\ 0 \\ 1 \end{bmatrix}.$$

Thus, we iterate $d_1 - d_2 + 1$ times, dividing the original polynomial until our largest degree is smaller than $d_2$. To simulate polynomial division, which involves multiplying the divisor by the additive inverse, we take the row with the largest degree and compute the Kronecker product with the divisor vector. We subtract the correct portion.

In [12]:
def moduloMats(divPoly,deg):
    # build dividend polynomial matrix
    divMat = np.eye(deg+1)
    
    # divisor vector
    cffs = sym_allcoeffs(divPoly)
    divDeg = len(cffs)-1

    for i in range(deg-divDeg+1):
        idx = deg-i
        row = divMat[idx,:]
        prod = np.outer(cffs,row)
        divMat[idx-divDeg:idx+1,:] -= prod
        
    return divMat[:divDeg,:]

## ax^3+bx^2+cx+d (mod x^2+1) = (c-a)x + (d-b)
print(moduloMats(M,3))

[[ 1.  0. -1.  0.]
 [ 0.  1.  0. -1.]]


In [13]:
# evaluates our input degree 3 (size 4) in the polynomial modulo M,m
M1 = moduloMats(M,3)
M2 = moduloMats(m,3)

C,A,_ = toomCookMats(2,2)
A1 = np.dot(A,M1)
A2 = np.dot(A,M2)
AA = np.vstack([A1,A2])

# evaluates the result degree 2n-1 in poly. modulo M,m again
M3 = moduloMats(M,2)
M4 = moduloMats(m,2)

C1 = np.dot(M3,C)
C2 = np.dot(M4,C)
CC = scila.block_diag(C1,C2)

# linear combinations by Berzouts identity for the CRT
E1 = berzotMats(N,2)
E2 = berzotMats(n,2)
E = np.hstack([E1,E2])
EC = np.dot(E,CC)

In [14]:
f = np.ones(4)
g = np.ones(4)
y = EC @ (np.dot(AA,f) * np.dot(AA,g))
print(y)

[4. 4. 4. 4.]


In [15]:
def cyclicWinoMats(d,RR,x):
    m1 = x**(d//2) + 1
    m2 = x**(d//2) - 1
    
    d1,d2 = d//2,d//2
    
    # evaluates our input d-point, or (d-1)-degree
    # in the polynomial modulo M,m
    M1 = moduloMats(m1,d-1)
    M2 = moduloMats(m2,d-1)

    # split nesting uses Agarawal to derive larger small convs
    C,A,_ = toomCookMats(d1,d1)
    A1 = np.dot(A,M1)
    A2 = np.dot(A,M2)
    AA = np.vstack([A1,A2])

    # evaluates the result degree (2n-1)-1 in poly. modulo M,m again
    M3 = moduloMats(m1,2*d1-2)
    M4 = moduloMats(m2,2*d2-2)

    C1 = np.dot(M3,C)
    C2 = np.dot(M4,C)
    CC = scila.block_diag(C1,C2)

    # linear combinations by Berzouts identity for the CRT
    N,n = extEucAlg(m1,m2,RR,d)
    E1 = berzotMats(N,d1)
    E2 = berzotMats(n,d2)
    E = np.hstack([E1,E2])
    EC = np.dot(E,CC)
    
    # [C,A,B]
    return [EC,AA,AA]

In [16]:
d = 4 # generate a d-point cyclic convolution
assert(d % 2 == 0)

[C,A,_] = cyclicWinoMats(d,RR,x)
v = np.ones(d)
w = np.ones(d)

y = C @ (np.dot(A,v) * np.dot(A,w))
print(y)

[4. 4. 4. 4.]


### Uneven Winograd Cyclic Conv Generator

In [17]:
ppp = x**16-1
ppp1 = x-1

print(ppp)
print(ppp/ppp1)

x**16 - 1
x**15 + x**14 + x**13 + x**12 + x**11 + x**10 + x**9 + x**8 + x**7 + x**6 + x**5 + x**4 + x**3 + x**2 + x + 1


### General Winograd Convolution Generator

We assuming the modulo spaces matches the degree of the polynomial

In [18]:
def _buildModuloMat(modPoly,divMat):
    # divisor as vector of coefficients
    deg = len(divMat)-1
    poly = sym_allcoeffs(modPoly)
    polyDeg = len(poly)-1

    # long division
    for i in range(deg-polyDeg+1):
        idx = deg-i
        row = divMat[idx,:]
        prod = np.outer(poly,row)
        divMat[idx-polyDeg:idx+1,:] -= prod
        
    return divMat[:polyDeg,:]

# Input(polynomial modulo, 
# to-be evaluated d-degree polynomial - can be arbitrary
# if genericPoly is empty
#       [optional]generic starting polynomial)
# Output: Matrix of a polynomial of degree (d-1) in modulo space
# I.e. Evaluate (P(x) mod M) (mod m_i)
# Note: polynomial in modulo poly (d+1)-degree evals to d-degree
def buildModuloMat(modPoly,deg,genericPoly=None):
    if(genericPoly is None):
        # d-degree has d+1 coefficients
        # where ith variable goes to power x^i
        genericPoly = np.eye(deg+1)
    return _buildModuloMat(modPoly,genericPoly)

print(buildModuloMat(x+1,2))

[[ 1. -1.  1.]]


### Idempotent Evaluation

In the recovery of the polynomial CRT, we need the idempotent $e_1 = m_2n_2$ and $e=2 = m_1n_1$ to evaluate the polynomial. Thus, the resultant polynomial product $r(x)$ needs to be multiplied by $e_i$ and then evaluted in the original modulo space $M(x)$.

So to do so, we first 1) Multiplying the idempotent by a generic polynomial in its modulo space. With the new product polynomial, we then 2) Evaluate the slightly altered generic in modulo space N.

In [19]:
# Input(idompotent poly, to-be multiplied d-degree polynomial)
# Output: Matrix of polynomial product (no modulo)
def _buildMultMat(idePoly,deg):
    ideCoeff = sym_allcoeffs(idePoly)
    ideDeg = len(ideCoeff)-1
    
    # has same number of deg+1 variables, but power
    # will be extended to deg+ideDeg
    prodMat = np.zeros((deg+ideDeg+1,deg+1))
    
    # multiply by coefficient of x^i
    for i in range(ideDeg+1):
        prodMat[i:i+deg+1,:] += ideCoeff[i] * np.eye(deg+1)
    return prodMat

# Input(idempotent polynomial coeffs,
#        large modulo polynomial,small mod space poly)
# Output: Matrix of evaluated product in modulo space m
def buildIdeMat(idePoly,M,mi):
    deg_m = sym_deg(mi)
    deg_M = sym_deg(M)
    # recall given (d+1)-degree mod poly, evals d-degree poly
    prodMat = _buildMultMat(idePoly,deg_m-1)
    ideMat = buildModuloMat(M,_,prodMat)
    return ideMat

print(buildIdeMat(-(x+x**2), x**3+2*x**2+2*x+1,x**2+x+1))

[[ 0.  1.]
 [-1.  2.]
 [-1.  1.]]


In [20]:
m1 = x+1
m2 = x**2+x+1
M = m1 * m2
print(M,"||",m1,"||",m2)

print(buildModuloMat(m1,2))
print(buildModuloMat(m2,2))

print(buildIdeMat(x**2+x+1, M, m1))
print(buildIdeMat(-(x+x**2), M, m2))

# for C matrix for a Toom-2
print(buildModuloMat(m2,2))

x**3 + 2*x**2 + 2*x + 1 || x + 1 || x**2 + x + 1
[[ 1. -1.  1.]]
[[ 1.  0. -1.]
 [ 0.  1. -1.]]
[[1.]
 [1.]
 [1.]]
[[ 0.  1.]
 [-1.  2.]
 [-1.  1.]]
[[ 1.  0. -1.]
 [ 0.  1. -1.]]


In [21]:
def getBerzetPolys(M,m,RR,deg):
    # returns coefficients of N(x) and n(x) 
    [N,n,gcd] = RR.dup_gcdex(M, m)
    assert(len(gcd) == 1 and gcd.coeffs()[0] == 1)
    return [N,n]
    # return [sym_allcoeffs(N),sym_allcoeffs(n)]

In [22]:
def getProdPoly(polys):
    prod = 1
    for p in polys:
        prod *= p
    return prod

def winoMats(polys,RR,x):
    M = getProdPoly(polys)
    deg_M = sym_deg(M)
    
    AA = None
    CC = None
    EE = None
    first = True
    
    for mi in polys:
        deg_m = sym_deg(mi)
        
        # evaluate in modulo
        L = buildModuloMat(mi,deg_M-1)
        C,A,_ = toomCookMats(deg_m,deg_m)
        A = np.dot(A,L)
        AA = (A if first else np.vstack([AA,A]))

        # evaluates interpolation modulo
        L = buildModuloMat(mi,2*deg_m-2)
        C = np.dot(L,C)
        CC = (C if first else scila.block_diag(CC,C))

        # recovery by the CRT
        Mi = M/mi
        N,n = getBerzetPolys(Mi,mi,RR,deg_M)
        E = buildIdeMat(N*Mi,M,mi)
        EE = (E if first else np.hstack([EE,E]))
        first = False
            
    EC = np.dot(EE,CC)
    return [EC,AA,AA]

In [23]:
polynomials = np.asarray([x+1,x**2+x+1])
[C,A,_] = winoMats(polynomials,RR,x)

print(C)
print(A)

[[ 1.  1. -1.  0.]
 [ 1.  1. -2.  1.]
 [ 1.  0. -1.  1.]]
[[ 1. -1.  1.]
 [ 1.  0. -1.]
 [ 1. -1.  0.]
 [ 0.  1. -1.]]


In [24]:
# example 1
polynomials = np.asarray([x*(x**2+1),(x+1)*(x**2+x+1)])
[C,A,_] = winoMats(polynomials,RR,x)

print(C)
print(A)

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 5.00000000e-01  1.00000000e+00  1.66666667e-01 -1.66666667e-01  1.00000000e+00  0.00000000e+00 -2.00000000e+00  1.66666667e-01  3.33333333e-01 -3.00000000e+00]
 [ 5.00000000e-01  2.50000000e+00 -5.55111512e-17 -5.00000000e-01  5.00000000e+00 -1.50000000e+00 -2.00000000e+00  5.00000000e-01  5.00000000e-01 -6.00000000e+00]
 [ 1.00000000e+00  3.00000000e+00 -3.33333333e-01 -6.66666667e-01  8.00000000e+00 -1.50000000e+00 -2.50000000e+00  5.00000000e-01  5.00000000e-01 -6.00000000e+00]
 [ 1.50000000e+00  2.00000000e+00 -5.00000000e-01 -5.00000000e-01  7.00000000e+00 -1.50000000e+00 -2.00000000e+00  5.00000000e-01  5.00000000e-01 -6.00000000e+00]
 [ 1.50000000e+00  5.00000000e-01 -3.33333333e-01 -1.66666667e-01  3.00000000e+00 -1.50000000e+00 -5.00000000e-01  3.33333333e-01  1.66666667e-01 -3.00000000e+00]]
[[ 1.  0.  0.  0.  0.

### Using Winograd within Winograd

From example 1, we can break the smaller convolutions further

In [25]:
# break larger mod spaces into smaller ones
def binomialWinoMats(polys,RR,x):
    numPolys = len(polys)
    midPt = numPolys//2
    
    M = getProdPoly(polys)
    deg_M = sym_deg(M)
    
    AA = None
    CC = None
    EE = None
    first = True
    
    for i in range(2):
        # build larger
        mi = 1
        start = (0 if i == 0 else midPt)
        end = (midPt if i == 0 else numPolys)
        for j in range(start,end):
            mi *= polys[j] 
        deg_m = sym_deg(mi)
        
        # evaluate in modulo
        L = buildModuloMat(mi,deg_M-1)
        if(end-start > 1):
            [C,A,B] = binomialWinoMats(polys[start:end],RR,x)
        else:
            [C,A,B] = toomCookMats(deg_m,deg_m)
        A = np.dot(A,L)
        AA = (A if first else np.vstack([AA,A]))

        # evaluates interpolation modulo
        L = buildModuloMat(mi,C.shape[0]-1)
        C = np.dot(L,C)
        CC = (C if first else scila.block_diag(CC,C))

        # recovery by the CRT
        Mi = M/mi
        N,n = getBerzetPolys(Mi,mi,RR,deg_M)
        E = buildIdeMat(N*Mi,M,mi)
        EE = (E if first else np.hstack([EE,E]))
        first = False
            
    EC = np.dot(EE,CC)
    return [EC,AA,AA]

In [26]:
polynomials = np.asarray([x, (x**2+1),(x+1),(x**2+x+1),(x-1),(x+2)])
[C,A,B] = binomialWinoMats(polynomials,RR,x)

print(C)
print(A)

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.50000000e+00 -6.00000000e-01  2.00000000e-01  2.00000000e-01 -5.00000000e-01  7.95659834e-17 -6.66666667e-01  6.66666667e-01  5.55555556e-02  1.11111111e-02]
 [ 1.50000000e+00 -1.10000000e+00  7.00000000e-01 -3.00000000e-01 -2.50000000e-01 -6.66666667e-01 -3.33333333e-01  1.00000000e+00  1.38888889e-01  1.11111111e-02]
 [ 5.00000000e-01 -6.00000000e-01  7.00000000e-01 -8.00000000e-01 -5.00000000e-01 -3.33333333e-01 -1.07321559e-16  3.33333333e-01  2.22222222e-01  1.11111111e-02]
 [-1.00000000e+00  5.00000000e-01 -1.38777878e-17 -5.00000000e-01  2.50000000e-01 -8.32667268e-17  6.93889390e-17  2.31296463e-17  2.50000000e-01  0.00000000e+00]
 [-1.50000000e+00  1.10000000e+00 -7.00000000e-01  3.00000000e-01  2.50000000e-01 -7.49400542e-17  6.66666667e-01 -6.66666667e-01  1.94444444e-01 -1.11111111e-02]
 [-1.50000000e+00  6.0

In [27]:
polynomials = np.asarray([x**4+1,x+1,x-1,x**2+1])
[C,A,B] = binomialWinoMats(polynomials,RR,x)

print(C)
print(A)

[[ 3.75000000e-01  8.33333333e-02  8.33333333e-02 -2.08333333e-02 -2.08333333e-02 -2.16840434e-19  2.50000000e+00  1.25000000e-01  1.25000000e-01  2.50000000e-01  0.00000000e+00 -2.50000000e-01]
 [ 1.25000000e-01 -4.58333333e-01  2.70833333e-01  1.04166667e-01 -2.91666667e-02 -1.25000000e-02  4.50000000e+00 -1.25000000e-01  1.25000000e-01  2.50000000e-01 -2.50000000e-01  2.50000000e-01]
 [-6.25000000e-01  3.33333333e-01  3.33333333e-01 -2.08333333e-02 -2.08333333e-02  5.85469173e-18  1.50000000e+00  1.25000000e-01  1.25000000e-01 -2.50000000e-01  0.00000000e+00  2.50000000e-01]
 [-2.08333333e-01  2.91666667e-01  2.08333333e-02 -1.45833333e-01  2.08333333e-02  2.08333333e-02 -7.50000000e+00 -1.25000000e-01  1.25000000e-01 -2.50000000e-01  2.50000000e-01 -2.50000000e-01]
 [-3.75000000e-01 -8.33333333e-02 -8.33333333e-02  2.08333333e-02  2.08333333e-02  6.50521303e-19 -2.50000000e+00  1.25000000e-01  1.25000000e-01  2.50000000e-01  0.00000000e+00 -2.50000000e-01]
 [-1.25000000e-01  4.5833

In [28]:
y = C @ (np.dot(A,np.ones(8)) * np.dot(A,np.ones(8)))
print(y)

[8. 8. 8. 8. 8. 8. 8. 8.]


### Converting Modular to Linear

Given a linear convolution in a modular space $M(x)$, how can we recover the answer, i.e. no algrbaric ring?

First, the modular space degree must match the degree of the product

In [29]:
polys = np.asarray([x*(x**2+1),(x+1)*(x**2+x+1)])
f = np.random.random(4)
g = np.random.random(4)
y = np.convolve(f,g)

[C,A,B] = winoMats(polys,RR,x)
print(C)
print(A)
# [C,A,B] = binomialWinoMats(polynomials,RR,x)
M = getProdPoly(polys)
Mcoeffs = sym_allcoeffs(M)
f2 = np.append(f,np.zeros(2))
g2 = np.append(g,np.zeros(2))
_y = C @ (np.dot(A,f2) * np.dot(B,g2))
print(_y)
y2 = np.zeros(len(_y) + 1)
y2[:-1] += _y
y2[-len(Mcoeffs):] += f[-1]*g[-1] * Mcoeffs

print(y)
print(y2)

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 5.00000000e-01  1.00000000e+00  1.66666667e-01 -1.66666667e-01  1.00000000e+00  0.00000000e+00 -2.00000000e+00  1.66666667e-01  3.33333333e-01 -3.00000000e+00]
 [ 5.00000000e-01  2.50000000e+00 -5.55111512e-17 -5.00000000e-01  5.00000000e+00 -1.50000000e+00 -2.00000000e+00  5.00000000e-01  5.00000000e-01 -6.00000000e+00]
 [ 1.00000000e+00  3.00000000e+00 -3.33333333e-01 -6.66666667e-01  8.00000000e+00 -1.50000000e+00 -2.50000000e+00  5.00000000e-01  5.00000000e-01 -6.00000000e+00]
 [ 1.50000000e+00  2.00000000e+00 -5.00000000e-01 -5.00000000e-01  7.00000000e+00 -1.50000000e+00 -2.00000000e+00  5.00000000e-01  5.00000000e-01 -6.00000000e+00]
 [ 1.50000000e+00  5.00000000e-01 -3.33333333e-01 -1.66666667e-01  3.00000000e+00 -1.50000000e+00 -5.00000000e-01  3.33333333e-01  1.66666667e-01 -3.00000000e+00]]
[[ 1.  0.  0.  0.  0.

In [84]:
def recoverLinConv(polys,_y,fLast,gLast):
    M = getProdPoly(polys)
    Mcoeffs = sym_allcoeffs(M)
    y = np.zeros(len(_y) + 1)
    y[:-1] += _y
    y[-len(Mcoeffs):] += fLast * gLast * Mcoeffs
    return y

def moduloToLinear(polys,n,r,C,A,B):
    p = getProdPoly(polys)
    pc = sym_allcoeffs(p)
    C2 = np.zeros((len(pc),C.shape[1]+1))
    C2[:C.shape[0],:C.shape[1]] = C.copy()
    C2[:,C.shape[1]] = pc.copy()
    A2 = np.vstack([A,np.zeros(A.shape[1])])
    A2 = A2[:,:r]
    B2 = np.vstack([B,np.zeros(B.shape[1])])
    B2 = B2[:,:n]
    A2[-1,-1] = 1
    B2[-1,-1] = 1
    return [C2,A2,B2]

In [85]:
n = r = 3
polys = np.asarray([x-1,x+1,x**2+1])
f = np.random.random(r)
g = np.random.random(n)
y = np.convolve(f,g)

[C,A,B] = winoMats(polys,RR,x)
f2 = np.append(f,np.zeros(1))
g2 = np.append(g,np.zeros(1))
_y = C @ (np.dot(A,f2) * np.dot(B,g2))
y2 = recoverLinConv(polys,_y,f[-1],g[-1])

print(y)
print(y2)

[0.09382974 0.19732385 0.6673629  0.68809783 0.26452165]
[0.09382974 0.19732385 0.6673629  0.68809783 0.26452165]


In [87]:
[C,A,B] = winoMats(polys,RR,x)
[C,A,B] = moduloToLinear(polys,n,r,C,A,B)
y3 = C @ (np.dot(A,f)*np.dot(B,g))
print(y3)

[0.09382974 0.19732385 0.6673629  0.68809783 0.26452165]


In [47]:
# give polynomials in cohorts
# polys: [[single polynomial],...[product, [...]], ]
def winoMats2(polys,RR,x):
    real_polys = np.asarray([])
    for poly_summary in polys:
        real_polys = np.append(real_polys,poly_summary[0])
    M = getProdPoly(real_polys)
    deg_M = sym_deg(M)
    
    AA = None
    CC = None
    EE = None
    first = True
    
    for mi_summary in polys:
        mi = mi_summary[0]
        deg_m = sym_deg(mi)
        
        # evaluate in modulo
        L = buildModuloMat(mi,deg_M-1)
        if(mi_summary.shape[0] > 1):
            C,A,B = winoMats2(mi_summary[1],RR,x)
        else:
            C,A,B = toomCookMats(deg_m,deg_m)
        A = np.dot(A,L)
        AA = (A if first else np.vstack([AA,A]))

        # evaluates interpolation modulo
        L = buildModuloMat(mi,C.shape[0]-1)
        C = np.dot(L,C)
        CC = (C if first else scila.block_diag(CC,C))

        # recovery by the CRT
        Mi = M/mi
        N,n = getBerzetPolys(Mi,mi,RR,deg_M)
        E = buildIdeMat(N*Mi,M,mi)
        EE = (E if first else np.hstack([EE,E]))
        first = False
            
    EC = np.dot(EE,CC)
    return [EC,AA,AA]

In [94]:
polys = np.asarray([
    np.asarray([x**4-1, np.asarray([
        np.asarray([x**2-1, np.asarray(
            [
                np.asarray([x-1]),
                np.asarray([x+1])
            ])
        ]),np.asarray([x**2+1])
    ]), ]),
    np.asarray([x**4+1])
])

f = np.ones(8)
g = np.ones(8)
y = np.convolve(f,g)

[C,A,B] = winoMats2(polys,RR,x)
# f2 = np.append(f,np.zeros(2))
# g2 = np.append(g,np.zeros(2))
_y = C @ (np.dot(A,f) * np.dot(B,g))
# y2 = recoverLinConv(polys,_y,f[-1],g[-1])

print(C)
print(A)
print(B)

print(_y)
# print(y2)

[[ 1.25000000e-01  1.25000000e-01  2.50000000e-01  0.00000000e+00 -2.50000000e-01  3.75000000e-01  8.33333333e-02  8.33333333e-02 -2.08333333e-02 -2.08333333e-02  0.00000000e+00  2.50000000e+00]
 [ 1.25000000e-01 -1.25000000e-01  2.50000000e-01 -2.50000000e-01  2.50000000e-01  1.25000000e-01 -4.58333333e-01  2.70833333e-01  1.04166667e-01 -2.91666667e-02 -1.25000000e-02  4.50000000e+00]
 [ 1.25000000e-01  1.25000000e-01 -2.50000000e-01  0.00000000e+00  2.50000000e-01 -6.25000000e-01  3.33333333e-01  3.33333333e-01 -2.08333333e-02 -2.08333333e-02  5.55111512e-18  1.50000000e+00]
 [ 1.25000000e-01 -1.25000000e-01 -2.50000000e-01  2.50000000e-01 -2.50000000e-01 -2.08333333e-01  2.91666667e-01  2.08333333e-02 -1.45833333e-01  2.08333333e-02  2.08333333e-02 -7.50000000e+00]
 [ 1.25000000e-01  1.25000000e-01  2.50000000e-01  0.00000000e+00 -2.50000000e-01 -3.75000000e-01 -8.33333333e-02 -8.33333333e-02  2.08333333e-02  2.08333333e-02  0.00000000e+00 -2.50000000e+00]
 [ 1.25000000e-01 -1.2500

In [95]:
polys = np.asarray([x+1,x-1,x**2+1,x**4+1])

f = np.ones(8)
g = np.ones(8)
y = np.convolve(f,g)

[C2,A2,B2] = winoMats(polys,RR,x)
# f2 = np.append(f,np.zeros(2))
# g2 = np.append(g,np.zeros(2))
_y2 = C2 @ (np.dot(A2,f) * np.dot(B2,g))
# y2 = recoverLinConv(polys,_y,f[-1],g[-1])

print(C2)
print(A2)
print(B2)

print(_y2)
# print(y2)

[[ 1.25000000e-01  1.25000000e-01  2.50000000e-01  0.00000000e+00 -2.50000000e-01  3.75000000e-01  8.33333333e-02  8.33333333e-02 -2.08333333e-02 -2.08333333e-02  0.00000000e+00  2.50000000e+00]
 [-1.25000000e-01  1.25000000e-01  2.50000000e-01 -2.50000000e-01  2.50000000e-01  1.25000000e-01 -4.58333333e-01  2.70833333e-01  1.04166667e-01 -2.91666667e-02 -1.25000000e-02  4.50000000e+00]
 [ 1.25000000e-01  1.25000000e-01 -2.50000000e-01  0.00000000e+00  2.50000000e-01 -6.25000000e-01  3.33333333e-01  3.33333333e-01 -2.08333333e-02 -2.08333333e-02  5.55111512e-18  1.50000000e+00]
 [-1.25000000e-01  1.25000000e-01 -2.50000000e-01  2.50000000e-01 -2.50000000e-01 -2.08333333e-01  2.91666667e-01  2.08333333e-02 -1.45833333e-01  2.08333333e-02  2.08333333e-02 -7.50000000e+00]
 [ 1.25000000e-01  1.25000000e-01  2.50000000e-01  0.00000000e+00 -2.50000000e-01 -3.75000000e-01 -8.33333333e-02 -8.33333333e-02  2.08333333e-02  2.08333333e-02  0.00000000e+00 -2.50000000e+00]
 [-1.25000000e-01  1.2500